In [13]:
# Notebook parameters. Values here are for development only and
# will be overridden when running via snakemake and papermill.
# cohort_id = "CD-NU_Gbadolite_gamb_2015_Q3"
cohort_id = "ML-2_Kati_colu_2014_Q3"
# cohort_id = 'CI-LG_Agneby-Tiassa_colu_2012'
cohorts_analysis = "20240924"
sample_sets = "3.0"
analysis_version = "2025.02.07"
dask_scheduler = "single-threaded"
contigs = ["2RL", "3RL", "X"]
min_cohort_size = 20
max_cohort_size = 50

In [ ]:
from pyprojroot import here

root = here()
%run -i {root}/workflow/notebooks/site-utils.py

# N.B., do not add the "remove-output" tag to this cell!!! If you do,
# the bokeh javascript libraries will not get loaded in the generated
# HTML page. The call to output_notebook() injects javascript in the
# cell output which triggers the bokeh javascript libraries to be loaded
# in the page.
output_notebook(hide_banner=True)

ERROR 1: PROJ: proj_create_from_database: Open of /home/alimanfoo/envs/selection-atlas/share/proj failed


# Cohort Foo

In [3]:
# Load cohorts to find sample query to select samples for this cohort.
cohort = gdf_cohorts.set_index("cohort_id").loc[cohort_id]
cohort

cohort_size                                                          27
country                                                            Mali
admin1_iso                                                         ML-2
admin1_name                                                  Koulikouro
admin2_name                                                        Kati
taxon                                                          coluzzii
year                                                               2014
quarter                                                               3
cohort_label                         Mali / Kati / coluzzii / 2014 / Q3
sample_query          cohort_admin2_quarter == 'ML-2_Kati_colu_2014_...
latitude                                                      12.875556
longitude                                                     -8.137778
h12_window_size                                                    1000
country_alpha2                                                  

In [4]:
# Load sample metadata for this cohort.
df_samples = ag3.sample_metadata(
    sample_sets=sample_sets, sample_query=cohort["sample_query"]
)
df_samples

,sample_id,partner_sample_id,contributor,country,location,year,month,latitude,longitude,sex_call,...,admin1_name,admin1_iso,admin2_name,taxon,cohort_admin1_year,cohort_admin1_month,cohort_admin1_quarter,cohort_admin2_year,cohort_admin2_month,cohort_admin2_quarter
0,AZ0292-C,MA7-36,Austin Burt,Mali,Kababougou,2014,8,12.89,-8.15,F,...,Koulikouro,ML-2,Kati,coluzzii,ML-2_colu_2014,ML-2_colu_2014_08,ML-2_colu_2014_Q3,ML-2_Kati_colu_2014,ML-2_Kati_colu_2014_08,ML-2_Kati_colu_2014_Q3
1,AZ0293-C,MA7-37,Austin Burt,Mali,Kababougou,2014,8,12.89,-8.15,F,...,Koulikouro,ML-2,Kati,coluzzii,ML-2_colu_2014,ML-2_colu_2014_08,ML-2_colu_2014_Q3,ML-2_Kati_colu_2014,ML-2_Kati_colu_2014_08,ML-2_Kati_colu_2014_Q3
2,AZ0294-C,MA7-4,Austin Burt,Mali,Kababougou,2014,8,12.89,-8.15,F,...,Koulikouro,ML-2,Kati,coluzzii,ML-2_colu_2014,ML-2_colu_2014_08,ML-2_colu_2014_Q3,ML-2_Kati_colu_2014,ML-2_Kati_colu_2014_08,ML-2_Kati_colu_2014_Q3
3,AZ0295-C,MA7-41,Austin Burt,Mali,Kababougou,2014,8,12.89,-8.15,F,...,Koulikouro,ML-2,Kati,coluzzii,ML-2_colu_2014,ML-2_colu_2014_08,ML-2_colu_2014_Q3,ML-2_Kati_colu_2014,ML-2_Kati_colu_2014_08,ML-2_Kati_colu_2014_Q3
4,AZ0296-C,MA7-44,Austin Burt,Mali,Kababougou,2014,8,12.89,-8.15,F,...,Koulikouro,ML-2,Kati,coluzzii,ML-2_colu_2014,ML-2_colu_2014_08,ML-2_colu_2014_Q3,ML-2_Kati_colu_2014,ML-2_Kati_colu_2014_08,ML-2_Kati_colu_2014_Q3
5,AZ0297-C,MA7-7,Austin Burt,Mali,Kababougou,2014,8,12.89,-8.15,F,...,Koulikouro,ML-2,Kati,coluzzii,ML-2_colu_2014,ML-2_colu_2014_08,ML-2_colu_2014_Q3,ML-2_Kati_colu_2014,ML-2_Kati_colu_2014_08,ML-2_Kati_colu_2014_Q3
6,AZ0298-C,MA7-8,Austin Burt,Mali,Kababougou,2014,8,12.89,-8.15,F,...,Koulikouro,ML-2,Kati,coluzzii,ML-2_colu_2014,ML-2_colu_2014_08,ML-2_colu_2014_Q3,ML-2_Kati_colu_2014,ML-2_Kati_colu_2014_08,ML-2_Kati_colu_2014_Q3
7,AZ0332-C,MA12-12,Austin Burt,Mali,Ouassorola,2014,8,12.90,-8.16,F,...,Koulikouro,ML-2,Kati,coluzzii,ML-2_colu_2014,ML-2_colu_2014_08,ML-2_colu_2014_Q3,ML-2_Kati_colu_2014,ML-2_Kati_colu_2014_08,ML-2_Kati_colu_2014_Q3
8,AZ0333-C,MA12-8,Austin Burt,Mali,Ouassorola,2014,8,12.90,-8.16,F,...,Koulikouro,ML-2,Kati,coluzzii,ML-2_colu_2014,ML-2_colu_2014_08,ML-2_colu_2014_Q3,ML-2_Kati_colu_2014,ML-2_Kati_colu_2014_08,ML-2_Kati_colu_2014_Q3
9,AZ0284-C,MA7-1,Austin Burt,Mali,Kababougou,2014,8,12.89,-8.15,F,...,Koulikouro,ML-2,Kati,coluzzii,ML-2_colu_2014,ML-2_colu_2014_08,ML-2_colu_2014_Q3,ML-2_Kati_colu_2014,ML-2_Kati_colu_2014_08,ML-2_Kati_colu_2014_Q3


In [5]:
# Determine collection dates.
df_collection_dates = (
    df_samples.groupby(["year", "month"])
    .size()
    .reset_index()
    .rename(columns={0: "count"})
)
df_collection_dates

,year,month,count
0,2014,8,27


In [6]:
# Determine first and last collection months.

min_month = df_collection_dates["month"].min()
max_month = df_collection_dates["month"].max()

if min_month < 0:
    start_month = end_month = None
else:
    start_month = pd.to_datetime(min_month, format="%m").month_name()
    end_month = pd.to_datetime(max_month, format="%m").month_name()

start_month, end_month

('August', 'August')

In [7]:
# Determine unique collection locations.
df_locations = df_samples[["location", "longitude", "latitude"]].drop_duplicates()
df_locations

,location,longitude,latitude
0,Kababougou,-8.15,12.89
7,Ouassorola,-8.16,12.90
21,Tieneguebougou,-8.08,12.81


In [8]:
# Extract provenance information about the samples.

df_contributors = df_samples[["release", "sample_set", "contributor"]].drop_duplicates()
df_contributors["study"] = df_contributors.apply(
    lambda v: "Ag1000G" if v["sample_set"].startswith("AG1000G") else "TODO",
    axis="columns",
)
df_contributors["release"] = df_contributors["release"].apply(lambda v: f"Ag{v}")
df_contributors.rename(
    columns={
        "contributor": "Contributor",
        "study": "Study",
        "release": "Data release",
        "sample_set": "Sample set",
    },
    inplace=True,
)
df_contributors.set_index(["Contributor", "Study", "Data release"], inplace=True)
df_contributors

,,,Sample set
Contributor,Study,Data release,
Austin Burt,Ag1000G,Ag3.0,AG1000G-ML-A


In [9]:
# Construct a paragraph with summary information about the samples in this cohort.

n_locations = len(df_locations)

summary_info = f"""This cohort comprises {cohort["cohort_size"]:,} samples from the 
*{cohort["taxon"]}* taxon, collected from {n_locations} locations within the administrative 
division of {cohort["admin2_name"]}, {cohort["admin1_name"]}, {cohort["country"]}."""

if start_month and start_month == end_month:
    summary_info += f""" Collections were made in {start_month} {cohort["year"]}."""
elif start_month:
    summary_info += f""" Collections were made between {start_month} and {end_month} in {cohort["year"]}."""
else:
    summary_info += f""" Collections were made in {cohort["year"]}."""

display(Markdown(summary_info))

This cohort comprises 27 samples from the 
*coluzzii* taxon, collected from 3 locations within the administrative 
division of Kati, Koulikouro, Mali. Collections were made in August 2014.

## Selection scans

In [10]:
# load signals to overlay on H12 plots.

dfs = []
for contig in contigs:
    df = load_cohort_signals(contig=contig, cohort_id=cohort_id)
    dfs.append(df)

df_signals = pd.concat(dfs)

# Add extra columns to help with overlaying signals on plots.
df_signals["bottom"] = 0
df_signals["top"] = 1

df_signals

,cohort_id,contig,gcenter,pcenter,delta_i,stat_max,gpos_max,ppos_max,focus_gstart,focus_gstop,...,skew,decay_left,decay_right,baseline,aic,bic,rss,constant_aic,bottom,top
0,ML-2_Kati_colu_2014_Q3,2RL,56.921394,28460698,3781,0.199588,56.942336,28471168,56.851603,56.956290,...,0.500000,0.279165,0.139583,0.026712,-19901.648084,-19873.507928,0.127284,-16119.799861,0,1
1,ML-2_Kati_colu_2014_Q3,2RL,81.883266,40941634,2402,0.117284,81.947584,40973792,81.798233,81.957842,...,0.094657,0.340132,0.298303,0.025335,-16777.165198,-16750.235943,0.048725,-14375.109909,0,1
2,ML-2_Kati_colu_2014_Q3,2RL,120.868338,64204801,2899,0.927984,120.809912,64175587,120.537650,121.050210,...,0.431277,1.322752,0.727485,0.013613,-7636.750854,-7610.095552,10.209972,-4737.336791,0,1
3,ML-2_Kati_colu_2014_Q3,2RL,166.443566,86992415,3944,0.244170,166.489816,87015539,166.239307,166.545701,...,0.499957,0.817035,0.408542,0.022162,-17271.475352,-17243.569303,0.291891,-13327.413252,0,1
4,ML-2_Kati_colu_2014_Q3,2RL,166.443566,86992415,3944,0.244170,166.489816,87015539,166.239946,166.545378,...,0.499983,0.814481,0.407250,0.022272,-17253.716883,-17225.818489,0.290156,-13309.165267,0,1
0,ML-2_Kati_colu_2014_Q3,3RL,57.094257,28547129,4759,0.198903,57.130386,28565193,56.992045,57.203479,...,-0.047854,0.408846,0.436888,0.021920,-18207.499649,-18179.857990,0.103724,-13448.108228,0,1
1,ML-2_Kati_colu_2014_Q3,3RL,141.740603,73052958,2787,0.112483,141.740607,73052959,141.714888,141.765025,...,0.037245,0.102862,0.097686,0.022081,-21668.292875,-21640.584459,0.019087,-18880.892901,0,1
0,ML-2_Kati_colu_2014_Q3,X,30.324885,15162443,4916,0.927984,30.302520,15151260,30.186625,30.547477,...,-0.343509,0.553039,0.890367,0.031082,-11414.210712,-11386.399904,5.075485,-6498.114251,0,1


In [ ]:
# load window sizes
with open(here() / h12_calibration_dir / f"{cohort_id}.yaml") as h12_calibration_file:
    h12_calibration_params = yaml.safe_load(h12_calibration_file)
h12_window_size = h12_calibration_params["h12_window_size"]
with open(here() / g123_calibration_dir / f"{cohort_id}.yaml") as g123_calibration_file:
    g123_calibration_params = yaml.safe_load(g123_calibration_file)
g123_window_size = g123_calibration_params["g123_window_size"]

if cohort.taxon == "arabiensis":
    phasing_analysis = "arab"
else:
    phasing_analysis = "gamb_colu"

ihs_window_size = 100


def plot_h12_g123_ihs_tracks(
    contig,
    df_signals,
    sizing_mode="stretch_width",
    show=False,
    width=800,
    track_height=150,
    genes_height=90,
):
    sample_query = cohort["sample_query"]

    fig1 = ag3.plot_h12_gwss_track(
        contig=contig,
        window_size=h12_window_size,
        analysis=phasing_analysis,
        sample_sets=sample_sets,
        sample_query=sample_query,
        min_cohort_size=min_cohort_size,
        max_cohort_size=max_cohort_size,
        sizing_mode=sizing_mode,
        show=show,
        width=width,
        height=track_height,
    )
    fig1.xaxis.visible = False

    if not df_signals.empty:
        df = df_signals.query("contig == @contig")
        center_xs = [np.array([row.pcenter, row.pcenter]) for idx, row in df.iterrows()]
        center_ys = [np.array([0, 1]) for idx, row in df.iterrows()]
        source = bkmod.ColumnDataSource(
            data={
                "cohort": df.cohort_id,
                "contig": df.contig,
                "score": df.delta_i.astype(int),
                "peak_start": df.span2_pstart,
                "peak_stop": df.span2_pstop,
                "focus_start": df.focus_pstart,
                "focus_stop": df.focus_pstop,
                "center_xs": center_xs,
                "center_ys": center_ys,
                "bottom": df.bottom,
                "top": df.top,
            }
        )
        quad = fig1.quad(
            bottom="bottom",
            top="top",
            left="peak_start",
            right="peak_stop",
            source=source,
            color="gray",
            alpha=0.1,
            line_width=1,
        )
        # TODO this is unused, need to check all good here?
        # quad2 = fig1.quad(
        #     bottom="bottom",
        #     top="top",
        #     left="focus_start",
        #     right="focus_stop",
        #     source=source,
        #     color="red",
        #     alpha=0.4,
        # )
        glyph = bkmod.MultiLine(
            xs="center_xs",
            ys="center_ys",
            line_color="red",
            line_width=2,
            line_alpha=0.8,
        )
        fig1.add_glyph(source, glyph)

        hover = bkmod.HoverTool(
            tooltips=[
                ("Cohort", "@cohort"),
                ("Score", "@score"),
                ("Focus", "@focus_start{,} - @focus_stop{,}"),
            ],
            renderers=[quad],
        )

        fig1.add_tools(hover)

    fig2 = ag3.plot_g123_gwss_track(
        contig=contig,
        window_size=g123_window_size,
        sites=phasing_analysis,
        site_mask=phasing_analysis,
        sample_sets=sample_sets,
        sample_query=sample_query,
        min_cohort_size=min_cohort_size,
        max_cohort_size=max_cohort_size,
        sizing_mode=sizing_mode,
        width=width,
        height=track_height,
        show=show,
        title="",
        x_range=fig1.x_range,
    )
    fig2.xaxis.visible = False

    fig3 = ag3.plot_ihs_gwss_track(
        contig=contig,
        window_size=ihs_window_size,
        analysis=phasing_analysis,
        sample_sets=sample_sets,
        sample_query=sample_query,
        min_cohort_size=min_cohort_size,
        max_cohort_size=max_cohort_size,
        sizing_mode=sizing_mode,
        width=width,
        height=track_height,
        show=show,
        title="",
        x_range=fig1.x_range,
    )
    fig3.xaxis.visible = False

    fig4 = ag3.plot_genes(
        region=contig,
        show=show,
        sizing_mode=sizing_mode,
        width=width,
        height=genes_height,
        x_range=fig1.x_range,
    )

    fig = bklay.gridplot(
        [fig1, fig2, fig3, fig4],
        ncols=1,
        toolbar_location="above",
        merge_tools=True,
        sizing_mode=sizing_mode,
    )
    return fig

In [ ]:
for contig in contigs:
    display(HTML(f"<h3>Chromosome {contig}</h3>"))

    fig = plot_h12_g123_ihs_tracks(
        contig=contig,
        df_signals=df_signals,
    )
    bkplt.show(fig)

Load haplotypes:   0%|          | 0/504 [00:00<?, ?it/s]

## Sampling information

In [ ]:
from ipyleaflet import Map, Marker, basemaps
from ipywidgets import HTML

center = cohort[["latitude", "longitude"]].to_list()
m = Map(center=center, zoom=9, basemap=basemaps.OpenTopoMap)

df = (
    df_samples[["latitude", "longitude", "taxon"]]
    .groupby(["latitude", "longitude", "taxon"])
    .size()
    .to_frame()
    .rename(columns={0: "count"})
    .reset_index()
)

for coh_id, row in df.iterrows():
    lat, long = row[["latitude", "longitude"]]

    if row["taxon"] == "gambiae":
        color = "red"
    elif row["taxon"] == "coluzzii":
        color = "cadetblue"
    elif row["taxon"] == "arabiensis":
        color = "lightgreen"
    else:
        color = "gray"

    marker = Marker(location=(lat, long), draggable=False, opacity=0.7, color=color)
    m.add_layer(marker)
    message2 = HTML()
    message2.value = f"n = {row['count']}"
    marker.popup = message2

display(m)

In [ ]:
if min_month >= 0:
    # For this cohort we have month data, so show a breakdown of sample
    # numbers by location and month.

    # Construct a pivot table counting samples.
    df_loc_dt = df_samples.pivot_table(
        index=["location", "longitude", "latitude"],
        columns="month",
        values="sample_id",
        aggfunc="count",
        fill_value=0,
    )

    # Tidy up the columns using a multi index.
    df_loc_dt.reset_index(inplace=True)
    cols = pd.MultiIndex.from_tuples(
        [("Location", "Name"), ("Location", "Longitude"), ("Location", "Latitude")]
        + [
            ("Date", pd.to_datetime(x, format="%m").month_name())
            for x in df_loc_dt.columns[3:]
        ],
    )
    df_loc_dt.columns = cols

else:
    # For this cohort we do not have month data, so show a breakdown of sample
    # numbers by location only.

    # Construct a pivot table counting samples.
    df_loc_dt = df_samples.groupby(["location", "longitude", "latitude"]).agg(
        {"sample_id": "count"}
    )

    # Tidy up the columns using a multi index.
    df_loc_dt.reset_index(inplace=True)
    cols = pd.MultiIndex.from_tuples(
        [
            ("Location", "Name"),
            ("Location", "Longitude"),
            ("Location", "Latitude"),
            ("Date", cohort.year),
        ]
    )
    df_loc_dt.columns = cols

# Style the table.
df_loc_dt_styled = (
    df_loc_dt.style.format(precision=3)
    .set_caption("Number of samples collected.")
    .hide()
)
display(df_loc_dt_styled)

## Diagnostics
### H12 calibration

In [ ]:
display(Markdown(f"Selected window size: **{h12_window_size:,}**"))

window_sizes = (100, 200, 500, 1000, 2000, 5000, 10000, 20000)

ag3.plot_h12_calibration(
    contig=h12_calibration_contig,
    analysis=phasing_analysis,
    sample_sets=sample_sets,
    sample_query=cohort["sample_query"],
    min_cohort_size=min_cohort_size,
    max_cohort_size=max_cohort_size,
    window_sizes=window_sizes,
);

### G123 Calibration

In [ ]:
display(Markdown(f"Selected window size: **{g123_window_size:,}**"))

ag3.plot_g123_calibration(
    contig=h12_calibration_contig,
    sites=phasing_analysis,
    site_mask=phasing_analysis,
    sample_sets=sample_sets,
    sample_query=cohort["sample_query"],
    min_cohort_size=min_cohort_size,
    max_cohort_size=max_cohort_size,
    window_sizes=window_sizes,
);

## Data sources

In [ ]:
df_sources_style = df_contributors.style.set_caption(
    "MalariaGEN Vector Observatory partners, studies and sample sets contributing data for this cohort."
)
df_sources_style